# Demystifying Decorators

## Rick Copeland

(and understanding functions and closures just a little bit more)

# Use Case #1: Aspect-Oriented Programming

AKA: "Doing something before/after a function is called"

## Examples:

* In concurrent programming, holding a lock (preventing other threads/processes/coroutines from running the same code at the same time)
* Ensuring that the current user/request/etc. has sufficient permissions to do what you're trying to do
* Logging, profiling, etc.

# Locking without decorators

```python
def transfer(amount, account1, account2):
    with account_transfers:
        if account1.balance > amount:
            account1.balance -= amount
            account2.balance += amount
        else:
            raise AccountOverdrawnError()
```

# Locking without decorators (nor `with`)

```python
def transfer(amount, account1, account2):
    account_transfers.acquire()
    try:
        if account1.balance > amount:
            account1.balance -= amount
            account2.balance += amount
        else:
            raise AccountOverdrawnError()
    finally:
        account_transfers.release()
```

# Access control

```python
def assert_permission(*permissions):
    for permission in permissions:
        if not current_user.has_permission(permission):
            raise Forbidden()
```

# Access control (without decorators)

```python
def code_that_accesses_private_data(...):
    assert_permission('account-manager')
    ... # other stuff
    
def other_unsafe_code(...):
    # oops! forgot the assert_permission call
    security_vulnerability_here()
```

# Access control (with decorators)

```python
@require_permission('account-manager')
def code_that_accesses_private_data(...):
    ... # other stuff
    
@require_permission('account-manager')
def other_unsafe_code(...):
    security_vulnerability_here()
```

* Decorators are easier to visually audit than function body code (do all my functions/views/etc. have a permission decorator?)

# Use Case #2: Side effects ("function registry")

## Flask without decorators

```python
def get_home():
    return jsonify({'page': 'Home'})

def get_about():
    return jsonify({'name': 'Rick Copeland'})

app.add_url_rule("/", view_func=get_home)
app.add_url_rule('/about', view_func=get_about)
```

## Flask with decorators (99% of folks use this)


```python
@app.route('/')
def get_home():
    return jsonify({'page': 'Home'})

@app.route('/about')
def get_about():
    return jsonify({'name': 'Rick Copeland'})
```

# So how does it all work? First, the syntax:

This:

```python
@something
def my_function():
    ...
```

is just fancy syntax for this:

```python
def my_function():
    ...
my_function = something(my_function)
```

# A brief aside: closures make this all possible

In [1]:
def make_adder(x):
    def add(y):
        return x + y
    return add

add39 = make_adder(39)

In [2]:
add39(6)

45

In [3]:
add12 = make_adder(12)
add12(6)

18

# Python Scoping

In [4]:
a = 10
b = 20
c = 30
def outer_function():
    a = 'ten'
    def inner_function():
        b = 'twenty'
        print('in inner function a=', a, 'b=', b, 'c=', c)
    print('in outer function (before inner), a=', a, 'b=', b, 'c=', c)
    inner_function()
    print('in outer function (after inner), a=', a, 'b=', b, 'c=', c)
print('in global level (before outer), a=', a, 'b=', b, 'c=', c)
outer_function()
print('in global level (before outer), a=', a, 'b=', b, 'c=', c)
              

in global level (before outer), a= 10 b= 20 c= 30
in outer function (before inner), a= ten b= 20 c= 30
in inner function a= ten b= twenty c= 30
in outer function (after inner), a= ten b= 20 c= 30
in global level (before outer), a= 10 b= 20 c= 30


# Python scoping precendence

* **L**ocal variables
* **E**nclosing (nested, still local) variables
* **G**lobal variables
* **B**uiltin variables (list, set, int, str, etc.)

# Let's build a decorator

Start simple: log the entry/exit of a function

In [5]:
def log_calls(func):
    def wrapper(*args, **kwargs):
        print(f'Calling {func.__name__}(args={args}, kwargs={kwargs})')
        result = func(*args, **kwargs)
        print(f' ==> {result}')
    return wrapper

Desired syntax:

In [6]:
@log_calls
def func1(a, b):
    print(f'func1({a}, {b})')
    return a + b
    
@log_calls
def func2(c, d):
    print(f'func2({c}, {d})')
    return c - d

In [7]:
func1(1, 2)
func2(3, 4)

Calling func1(args=(1, 2), kwargs={})
func1(1, 2)
 ==> 3
Calling func2(args=(3, 4), kwargs={})
func2(3, 4)
 ==> -1


Remember that this:

```python
@log_calls
def func1(a, b):
    print(f'func1({a}, {b})')
    return a + b
```

... really means this:

```python
def func1(a, b):
    print(f'func1({a}, {b})')
    return a + b

func1 = log_calls(func1)   # we call this "decorating" func1
```

... so what's the definition of log_calls?

# General decorator pattern: wrapping functions
        
```python
def decorator(function_being_decorated):
    def wrapper(*args, **kwargs):  # "general purpose" function signature
        # ... things to do before the function is called ...
        result = function_being_decorated(*args, **kwargs)
        # ... things to do after the function is called ...
        return result
    return wrapper 
```

As applied to our example...

In [1]:
def log_calls(function):
    def wrapper(*args, **kwargs):
        print(f'Calling {function.__name__}(args={args}, kwargs={kwargs})')
        result = function(*args, **kwargs)
        print(f' ==> {result}')
        return result
    return wrapper

In [9]:
@log_calls
def func1(a, b):
    "This is func1"
    print(f'func1({a}, {b})')
    return a + b
    
@log_calls
def func2(c, d):
    "This is func2"
    print(f'func2({c}, {d})')
    return c - d

In [10]:
func1(1, 2)
func2(3, 4)

Calling func1(args=(1, 2), kwargs={})
func1(1, 2)
 ==> 3
Calling func2(args=(3, 4), kwargs={})
func2(3, 4)
 ==> -1


# Success! (almost)

## No decoration

In [11]:
def func1(a, b):
    "This is func1"
    print(f'func1({a}, {b})')
    return a + b

help(func1)

Help on function func1 in module __main__:

func1(a, b)
    This is func1



In [12]:
print(func1)

<function func1 at 0x7fbb9c5c3ca0>


Great introspection/documentation, decent name

# Success! (almost)

## With decoration

In [13]:
@log_calls
def func1(a, b):
    "This is func1"
    print(f'func1({a}, {b})')
    return a + b
help(func1)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [14]:
print(func1)

<function log_calls.<locals>.wrapper at 0x7fbb9c584160>


Horrible introspection/documentation, WTF name?!

# Fixing introspection

In [15]:
from functools import wraps

def log_calls(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        print(f'Calling {function.__name__}(args={args}, kwargs={kwargs})')
        result = function(*args, **kwargs)
        print(f' ==> {result}')
    return wrapper

@log_calls
def func1(a, b):
    "This is func1"
    print(f'func1({a}, {b})')
    return a + b
help(func1)

Help on function func1 in module __main__:

func1(a, b)
    This is func1



Adding the `@wraps` decorator _copies_ all the introspection information from the _function being wrapped_ to the _wrapper function_:

In [16]:
print(func1)

<function func1 at 0x7fbb9c584790>


In [17]:
func1(1,2)

Calling func1(args=(1, 2), kwargs={})
func1(1, 2)
 ==> 3


<img src="data/img/yo-dawg-decorators.jpg">

# Use Case #2: Side effects ("function registry")

Something like Flask needs to keep a mapping (like a `dict`) between a URL pattern and a Python function (a 'view'). 

Simpler version of Flask (Teacup?)

```python
@route('/')
def get_home():
    return {'page': 'Home'}

@route('/about')
def get_about():
    return {'name': 'Rick Copeland'}
```

# Let's build it!

First, remember to translate the syntax:

```python
@route('/')
def get_home():
    return {'page': 'Home'}
```
... really means ...

```python

_temp = route('/')
def get_home():
    return {'page': 'Home'}
get_home = _temp(get_home)
```

# Actual implementation

In this case, we **aren't** going to be creating a wrapper. 

Instead, we'll generate a **side-effect** of entering the function in a global registry of "routes"

In [18]:
global_routes_dict = {}

def route(url_pattern):
    "This is not a decorator! It is a *decorator factory*. It **builds** decorators!"
    def decorator(view_function):
        "This is the actual decorator (helpfully named!)"
        global_routes_dict[url_pattern] = view_function
        return view_function # notice we are not wrapping!
    return decorator

In [19]:
@route('/')
def get_home():
    return {'page': 'Home'}

@route('/about')
def get_about():
    return {'name': 'Rick Copeland'}


In [20]:
def teacup_dispatcher(url_path):
    view_function = global_routes_dict.get(url_path)
    if view_function is None:
        raise ValueError('404 Not Found')  # probably should use custom exception
    return view_function()

In [21]:
teacup_dispatcher('/')

{'page': 'Home'}

In [22]:
teacup_dispatcher('/about')

{'name': 'Rick Copeland'}

# Wrappers or decorator factories?

<img src="data/img/decorators-why-not-both.jpg">

# How about some (performance-sensitive) instrumentation?

## Problem: we need to know where our (high-traffic web-based) app is spending all its time

## Solution (?): Profile!

## The 'now you have 2 problems' problem: Profiling is _SLOW_

# HOW SLOW IS IT?

In [23]:
import re
import time

def find_foxes(text):
    return re.findall('fox', text)

In [24]:
%%timeit
find_foxes('the quick brown fox jumps over the lazy dog')

438 ns ± 8.05 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


... but with profiling turned on...

In [25]:
from cProfile import Profile
prof = Profile()

In [26]:
%%timeit
with prof:
    find_foxes('the quick brown fox jumps over the lazy dog')

1.54 µs ± 22.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# Profile data is great...

In [27]:
prof.print_stats(sort='time')

         56777777 function calls in 8.973 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  8111111    1.999    0.000    5.868    0.000 re.py:233(findall)
  8111111    1.849    0.000    2.569    0.000 re.py:289(_compile)
  8111111    1.676    0.000    7.544    0.000 <ipython-input-23-a9d52e7f3a05>:4(find_foxes)
  8111111    1.299    0.000    1.299    0.000 {method 'findall' of 're.Pattern' objects}
  8111111    1.044    0.000    1.429    0.000 cProfile.py:133(__exit__)
  8111111    0.720    0.000    0.720    0.000 {built-in method builtins.isinstance}
  8111111    0.385    0.000    0.385    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## ...but 3x or more slowdown is a non-starter...

# What if we sampled say - 5% of the calls?

In [28]:
from random import random

def sampling_profiler(prof, rate):
    def decorator(function):
        @wraps(function)
        def wrapper(*args, **kwargs):
            num = random()
            if num < rate:
                with prof:
                    return function(*args, **kwargs)
            else:
                return function(*args, **kwargs)
        return wrapper
    return decorator

# Really?!

Yes, I know that's **3** levels of nesting functions...

...with a gratuitous decorator thrown in.

It is nonetheless a pretty common (and Pythonic!) pattern. 

I'm very sorry.

# So anyway, it works pretty well...

In [29]:
prof = Profile()

@sampling_profiler(prof, 0.05)
def find_foxes(text):
    return re.findall('fox', text)

In [30]:
%%timeit
find_foxes('the quick brown fox jumps over the lazy dog')

672 ns ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [31]:
prof.print_stats(sort='time')

         2845129 function calls in 0.498 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   406447    0.113    0.000    0.323    0.000 re.py:233(findall)
   406447    0.100    0.000    0.139    0.000 re.py:289(_compile)
   406447    0.096    0.000    0.419    0.000 <ipython-input-29-d44eab210293>:3(find_foxes)
   406447    0.070    0.000    0.070    0.000 {method 'findall' of 're.Pattern' objects}
   406447    0.057    0.000    0.078    0.000 cProfile.py:133(__exit__)
   406447    0.039    0.000    0.039    0.000 {built-in method builtins.isinstance}
   406447    0.022    0.000    0.022    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# And if you want to actually make it faster...

In [32]:
prof = Profile()
re_fox = re.compile('fox')

@sampling_profiler(prof, 0.05)
def find_foxes(text):
    return re_fox.findall(text)

In [33]:
%%timeit
find_foxes('the quick brown fox jumps over the lazy dog')

336 ns ± 5.38 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [34]:
prof.print_stats(sort='time')

         1622564 function calls in 0.197 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   405641    0.066    0.000    0.066    0.000 {method 'findall' of 're.Pattern' objects}
   405641    0.060    0.000    0.126    0.000 <ipython-input-32-789a08d53b88>:4(find_foxes)
   405641    0.050    0.000    0.071    0.000 cProfile.py:133(__exit__)
   405641    0.020    0.000    0.020    0.000 {method 'disable' of '_lsprof.Profiler' objects}




... so if you are going to use a pattern more than once, pre-compile it...

# Thank you... any questions?

## Rick Copeland

## rick@arborian.com

(available for group Python, ML, and Data Science Training)